In [ ]:
%matplotlib inline 

# Intro to Data Analysis in Python

This tutorial provides an overview of common data anlysis tasks and packages in Python. "Intro to Programming in Python" is a pre-requisite, or working knowledge of data types and structures, loops and conditional statements, functions, reading and writing data. 
In this tutorial we will: 
- Import the packages needed. 
- Read in the data we're using as an example. 
- Clean and proces the data using different functions in the Pandas package. 
- Compute descriptive statistics for the variables of interest. 
- Display frequency and proportions tables. 
- Calculate correlations. 
- Conduct hypothesis tests. 
- Conduct regression analysis and regression diagnostics checks. 
- Visualize relations in the data. 


## Importing modules

Everything that we've done so far was based on functions from base Python. However, we will often need to import other packages which can handle more complex or specific tasks. For example, we may want to use a module that is able to better read and write csv data, such as the 'csv' module. To do that, we have to first import the module. For packages that are already installed, you can simply do that by typing 'import' and the name of the package. Many of the useful packages are already installed in Anaconda.

In [ ]:
import csv

But how do you know which packages are installed? If you open Anaconda Prompt, and you type "conda list", it will list all installed package. You can do this in any terminal/command prompt.

If a package is not installed, you can install it in the Anaconda Prompt with: `conda install csv`

## Reading data in different formats

The 'csv' module is already installed in Anaconda, so we can go ahead and import it. Let's read the file in csv format, recode missing values as NA, and write it out as a new clean.csv. The 'csv' module is very useful for manipulating large files that contain long text fields.  

In [ ]:
import csv
with open("clean.csv", "w") as outfile:
    writer=csv.writer(outfile)
    with open("mydataset.csv", "r") as infile:  # open the file for writing
        reader=csv.reader(infile)
        writer.writerow(next(reader))
        for row in reader:
            writer.writerow(row[0:6]+[(row[6].replace("missing", "NaN"))])
            

You can also read csv files, as well as other file formats using Pandas. Pandas is one of the main libraries for data analysis in Python. For those of you familiar with R, the data frames structure and Pandas will make it very easy to use. Let's see what we can do, by importing the clean.csv file that you just saved. 

In [ ]:
import pandas as pd # we import it as pd because it's easier to type
df=pd.read_csv("clean.csv")
df

## Processing data with Pandas

In [ ]:
df.head(5) # first 5 rows

In [ ]:
df.tail(3) # last 3 rows

In [ ]:
df.shape # how many rows and columns

In [ ]:
df.columns # the column names

In [ ]:
df["reelected"][0:5] # select a column, and a slice within it

In [ ]:
df.region.unique() # Unique values in a column

In [ ]:
# Subsetting data: create another data frame that only includes obswervations from the South and East. 
value_list=["South", "East"]
df_SE=df[df.region.isin(value_list)] # Replace this with df[~df.region...] to keep only those that don't meet the condition
df_SE.count()

In [ ]:
# Select only dataframes that meet multiple conditions:
df_restricted=df[(df['region']=="South") & (df["chamber"]=="S") & (df["reelected"]==0)]
df_restricted.head(5)

In [ ]:
# Group and aggregate 
grouped=df.groupby(["region", "chamber"])
aggregated=grouped.agg({"spent":['sum','mean', 'min'], 
                       'raised':['sum', 'mean', 'max']})
aggregated

## Descriptive statistics

Mean, median, 25th and 75th quartiles, min, max, number of missing observations, etc. 


In [ ]:
# For numeric variables in the dataset
df.describe()

In [ ]:
#Let's turn off scientific notation:
pd.options.display.float_format = '{:.2f}'.format
df.describe()

In [ ]:
# For all variables in the dataset
df.describe(include='all')

You can compute summary statistics on a single variable:

In [ ]:
df['spent'].describe()

You can also calculate one value of interest at a time:

In [ ]:
#The sum of all the money raised
df['raised'].sum()


In [ ]:
#Cumulative sum of the money raised 
df['raised'].cumsum()

In [ ]:
#Count the number of non-NA values
df['reelected'].count()


In [ ]:
#Minimum spending
df['spent'].min()

In [ ]:
#Maximum spending
df['spent'].max()

In [ ]:
#Mean spending
df['spent'].mean()

In [ ]:
#Median spending
df['spent'].median()

In [ ]:
#Skewness of spending values
df['spent'].skew()

In [ ]:
#Sample variance of spending
df['spent'].var()

In [ ]:
#Sample standard deviation of spending
df['spent'].std()

In [ ]:
#Kurtosis of spending values
df['spent'].kurt()

## Frequency and proportions tables
You can make al types of frequency tables in Pandas using the *crosstab* function.

In [ ]:
# One way table of frequencies
pd.crosstab(index=df["chamber"], columns="count")

In [ ]:
# One way table of proportions
pd.crosstab(index=df["chamber"],  # Make a crosstab
            columns="count",  # Name the count column
           normalize="all")  # Display as percentages. Options are all, index(row), column. 

In [ ]:
# Two way table of frequencies
pd.crosstab(df.chamber, df.reelected)

In [ ]:
# Two way table of frequencies
pd.crosstab(df.chamber, df.reelected, normalize="all") #proportion of total

In [ ]:
# Two way table of frequencies
pd.crosstab(df.chamber, df.reelected, normalize="index") #proportion of rows

In [ ]:
# Two way table of frequencies
pd.crosstab(df.chamber, df.reelected, normalize="columns") #proportion of columns

In [ ]:
# Two way table of frequencies
pd.crosstab(df.chamber, df.reelected, normalize="all", margins=True) # add row and column totals

In [ ]:
# Label the rows and columns
mytab = pd.crosstab(df.chamber, df.reelected, normalize="all", margins=True)   # Include row and column totals
mytab.columns = ["Reelect_0","Reelect_1","Reelect_2","Reelect_3","Reelect_4","Reelect_5","Row_total"]
mytab.index= ["House","Senate","Col_total"]

mytab

In [ ]:
import seaborn
heat_plot=seaborn.heatmap((mytab), annot=True) 
heat_plot.get_figure().savefig("heatmap_output.png", bbox_inches='tight')


## Correlation and covariance 
You can compute correlations in Pandas using the *corr* and *cov* functions. 

In [ ]:
df.corr()

In [ ]:
df.cov()

For only two variables:

In [ ]:
df["spent"].corr(df["raised"])

In [ ]:
df["spent"].cov(df["raised"])

## Hypothesis testing
For the next section we'll import the *scipy.stats* sub-module of *scipy*.


In [ ]:
import scipy.stats as stats

We can test if the population mean of data is likely to be equal to a given value (if observations are drawn from a Gaussian distributions of given population mean) by using the scipy.stats.ttest_1samp()function. It returns the T statistic, and the p-value. 

In [ ]:
stats.ttest_1samp(df['spent'], 320000)

Is the mean spending in the House and Senate different, and it the difference statistically significant? We can do a 2-sample t-test with scipy.stats.ttest_ind():  

In [ ]:
house_spent = df[df['chamber'] == 'H']['spent']
senate_spent = df[df['chamber'] == 'S']['spent']

print("House mean:", house_spent.mean(), "Senate mean:", senate_spent.mean(), stats.ttest_ind(house_spent, senate_spent))

## Regression analysis
We can use the *ols* function from the *statsmodels* module to conduct OLS regression analysis. 


In [ ]:
from statsmodels.formula.api import ols

### Simple linear regression. 

In [ ]:
model1 = ols("spent ~ raised", df).fit()
print(model1.summary())  

Saveing the models to txt or csv: 

In [ ]:
with open('model1_summary.txt', 'w') as f:
    f.write(model1.summary().as_text())

with open('model1_summary.csv', 'w') as f:
    f.write(model1.summary().as_csv())

Regression with dummy variables. 

In [ ]:
model2 = ols("spent ~ chamber", df).fit()      
print(model2.summary())  

In [ ]:
model3 = ols("spent ~ region", df).fit()      
print(model3.summary())

### Multiple regression

In [ ]:
model4 = ols("spent ~ raised+chamber", df).fit()
print(model4.summary()) 

### Regression diagnostics. 
You can compute regression diagnostics using different funcions in *statsmodels*. You can read more about these here: [http://www.statsmodels.org/dev/diagnostic.html](http://www.statsmodels.org/dev/diagnostic.html). For example: 

In [ ]:
#Cook's distance
import statsmodels.graphics.regressionplots as regplots
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12,8))
fig = regplots.influence_plot(model4, ax=ax, criterion="cooks")

In [ ]:
fig = plt.figure(figsize=(12,8))
fig = statsmodels.graphics.regressionplots.plot_partregress_grid(model4, fig=fig)




Plot the dependent variable and fitted values with confidence intervals vs. an independent variable, the residuals of the model vs. the chosen independent variable, a partial regression plot, and a CCPR plot. Use function to quickly checking modeling assumptions with respect to a single regressor.


In [ ]:

fig = plt.figure(figsize=(12,8))
fig = statsmodels.graphics.regressionplots.plot_regress_exog(model4, "raised", fig=fig)



Fitted values vs a chosen independent variable:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
fig = statsmodels.graphics.regressionplots.plot_fit(model4, "raised", ax=ax)

## Logistic regression
What is the probability that 

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import glm
from pandas.core import datetools
model5 = glm("chamber~ raised+spent+reelected", df, family=sm.families.Binomial()).fit()
print(model5.summary()) 

## Visualization
To display graphs inline in Jupyter notebooks make sure you add "%matplotlib inline" in the first cell. 

In [ ]:
%matplotlib inline 
#%matplotlib notebook

### Histograms, comparing two distributions. 

In [ ]:
import matplotlib.pyplot as plt
df_money=df[["raised","spent"]]
df_money.plot.hist(stacked=True, bins=50)

### Barplots


In [ ]:
agg2=grouped.agg({"spent":"mean", 
                  "raised":"mean"})
agg2.plot.bar()

### Scatterplot with linear fit line

In [ ]:
import numpy as np

x=df_money.raised.values
y=df_money.spent.values
fig, ax = plt.subplots()
fit = np.polyfit(x, y, deg=1)
ax.plot(x, fit[0] * x + fit[1], color='red')
ax.scatter(x, y)


Or using Seaborn:
    

In [ ]:
seaborn.regplot(x="raised", y="spent", data=df)

In [ ]:
seaborn.lmplot(x="raised", y="spent", hue="chamber", data=df)

Explore multiple regression relations with the module *seaborn*.

In [ ]:
seaborn.pairplot(df, vars=['spent', 'raised'], kind='reg', hue="chamber") 

## Additional resources


[Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)

[Statsmodels documentation](http://www.statsmodels.org/stable/index.html)

[Seaborn examples](https://seaborn.pydata.org/examples/)

